In [68]:
import os
import requests
import pandas as pd
import numpy as np
import time
from dotenv import  load_dotenv

In [69]:
# Load API Key
load_dotenv()
API_KEY = os.getenv("API_KEY")
print("API ending in ..."+str(API_KEY)[-5:-1]+" has been loaded succesfully" if API_KEY else "Not found")

API ending in ...osHJ has been loaded succesfully


In [70]:
# Get cash amount and interest earned
url = "https://live.trading212.com/api/v0/equity/account/cash"
headers = {"Authorization": API_KEY}
response = requests.get(url, headers=headers)
cash_data = response.json()

In [71]:
cash_df = pd.json_normalize(cash_data)
# Free cash includes cash in pie, substract pieCash from free for available cash
cash_df['free'] = cash_df['free'] - cash_df['pieCash']
cash_df = cash_df.drop(['result', 'pieCash', 'blocked'], axis=1)
cash_df.head()

,free,total,ppl,invested
0,1917.48,5789.07,104.49,3766.59


In [72]:
# Get original cash before interest
url = "https://live.trading212.com/api/v0/history/transactions"

query = {
  "limit": "10"
}

headers = {"Authorization": API_KEY}

response = requests.get(url, headers=headers, params=query)

data = response.json()
transactions = data['items']
next_page = data['nextPagePath']
print(data['items'])

while(next_page is not None):
    query['cursor'] = next_page[16:52]
    query['time'] = next_page[-24:]
    response = requests.get(url, headers=headers, params=query)
    data = response.json()
    trans = data['items']
    print(data['nextPagePath'])
    next_page = data['nextPagePath']
    print(trans)
    transactions.extend(trans)
    time.sleep(11)


[{'type': 'DEPOSIT', 'amount': 0.22, 'reference': '968bc230-7b59-438f-bfcc-bdebcec20689', 'dateTime': '2025-07-27T00:12:19.856Z'}, {'type': 'DEPOSIT', 'amount': 0.22, 'reference': '1925c283-f6a5-45e6-81d2-f97db50d65c6', 'dateTime': '2025-07-26T00:18:29.450Z'}, {'type': 'DEPOSIT', 'amount': 0.23, 'reference': 'de347c8b-133a-44ca-ab85-a7df87a54bef', 'dateTime': '2025-07-25T00:18:38.244Z'}, {'type': 'DEPOSIT', 'amount': 0.22, 'reference': 'e834ad41-21eb-49db-8b58-bb39bacb5342', 'dateTime': '2025-07-24T00:17:43.840Z'}, {'type': 'DEPOSIT', 'amount': 0.22, 'reference': '5d2511dd-7e94-4ad1-b8e4-86a96d07b651', 'dateTime': '2025-07-23T00:18:01.961Z'}, {'type': 'DEPOSIT', 'amount': 0.23, 'reference': '03bb348e-367a-4fdc-ad17-3123d59de8dc', 'dateTime': '2025-07-22T00:18:42.304Z'}, {'type': 'DEPOSIT', 'amount': 0.22, 'reference': 'a3a127c2-f103-42a6-b68b-2b0d389e4939', 'dateTime': '2025-07-21T00:12:33.207Z'}, {'type': 'DEPOSIT', 'amount': 0.22, 'reference': '570cf181-1a84-4cd3-831b-c37d9b58756a', 

In [75]:
url = "https://live.trading212.com/api/v0/history/transactions"

query = {
  "cursor": "00766c05-0ac6-414c-972c-cb9e5ce95445",
  "limit": "10",
  "time": "2025-01-23 16:43:17.296Z"
}

headers = {"Authorization": API_KEY}

response = requests.get(url, headers=headers, params=query)

data = response.json()
response

<Response [500]>

In [73]:
transactions_df = pd.DataFrame(transactions)
transactions_df.loc[(transactions_df['amount'] > 0)&(transactions_df['amount'] < 100), 'type'] = 'INTEREST'
transactions_df.tail()


,type,amount,reference,dateTime
121,INTEREST,0.01,a78b59ed-2e00-4777-81a8-67a349e9e7f5,2025-01-27T02:10:34.125Z
122,INTEREST,0.01,5ba10255-54c4-460d-96c6-4d2877b128df,2025-01-26T02:10:35.786Z
123,INTEREST,0.01,71b7aa17-60a1-4d8b-906a-09abb8a23ecc,2025-01-25T02:10:31.139Z
124,DEPOSIT,100.00,198596ac-acfd-4887-a333-cdff162af6bc,2025-01-24T16:39:35.250Z
125,INTEREST,0.01,4adfd051-b135-4716-8d86-d1ad1b462104,2025-01-24T02:10:30.960Z
